In [1]:
import pandas as pd
import lightfm
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k, reciprocal_rank, auc_score
from lightfm.cross_validation import random_train_test_split
from dataloading import DataLoader
from datapreparation_py import DataPreparation
from lightFM_adoptor import LightFMAdaptor
from cv_py import CrossValidation
from implicit.evaluation import ranking_metrics_at_k, train_test_split

c:\Users\781110104\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
%cd C:\Users\781110104\OneDrive - Genpact\Documents\VSCode

C:\Users\781110104\OneDrive - Genpact\Documents\VSCode


In [3]:
dl = DataLoader()
user_item_co_t = dl.import_data('TEREX', 'CO', 'df')

In [4]:
user_item_co_t_filtered = dl.remove_low_interact_items(user_item_co_t, 1)

In [5]:
user_item_filtered_log_t = dl.log_scale_df(user_item_co_t_filtered, 0.01)

In [6]:
dp = DataPreparation(user_item_filtered_log_t)

In [7]:
interactions_csr, weights_csr = dp.get_interaction_data()

In [8]:
user_features = ['country', 'brand', 'currency']
item_features = ['group2', 'movement_code', 'cost_class']
feature_input = dp.get_feature_data(user_features=user_features, item_features=item_features)

In [9]:
user_features = ['country']
item_features = ['group2']
userf_country, itemf_group2 = dp.get_feature_data(user_features=user_features, item_features=item_features)

In [10]:
user_features = ['brand']
item_features = ['movement_code']
userf_brand, itemf_move = dp.get_feature_data(user_features=user_features, item_features=item_features)

In [11]:
user_features = ['currency']
item_features = ['cost_class']
userf_curr, itemf_cost = dp.get_feature_data(user_features=user_features, item_features=item_features)

In [40]:
user_features = ['country', 'brand']
item_features = ['group2', 'movement_code']
userf_country_brand, itemf_group2_move = dp.get_feature_data(user_features=user_features, item_features=item_features)

In [41]:
user_features = ['country', 'currency']
item_features = ['group2', 'cost_class']
userf_country_curr, itemf_group2_cost = dp.get_feature_data(user_features=user_features, item_features=item_features)

In [42]:
user_features = ['brand', 'currency']
item_features = ['movement_code', 'cost_class']
userf_brand_curr, itemf_move_cost = dp.get_feature_data(user_features=user_features, item_features=item_features)

In [46]:
def unlist_df(my_list):
    for i in range(len(my_list)):
        if i == 0:
            ret_df = my_list[i].copy()
        else:
            ret_df = pd.concat([ret_df, my_list[i].copy()])
    return ret_df

In [12]:
userf, itemf = feature_input

In [13]:
train_i, test_i = train_test_split(interactions_csr, 0.8, 22)
train_w, test_w = train_test_split(weights_csr, 0.8, 22)

In [14]:
cal_w, val_w = train_test_split(train_w, 0.8, 22)

In [15]:
cv = CrossValidation(5)

Testing the model with simple parameter tuning:

First with WARP loss:

In [104]:
# step 2: take parameters and tune over all feature combinations lrtemp: 0.003, 0.007, 0.01, to add with regtemp: 0.0001
space_FM_warp = {'factors' : [64], 'regularization' : [0.0001, 0.0003, 0.0007, 0.001], 'learning_rate' : [0.003, 0.007, 0.01, 0.03], 'iterations' : [15], 'loss' : ['warp'], 'max_sampled' : [20, 30, 35, 40]}

uf_combi = [None, userf_country, userf_brand, userf_curr, userf_country_brand, userf_country_curr, userf_brand_curr, userf]
if_combi = [None, itemf_group2, itemf_move, itemf_cost, itemf_group2_move, itemf_group2_cost, itemf_move_cost, itemf]

uf_names = ['None', 'userf_country', 'userf_brand', 'userf_curr', 'userf_country_brand', 'userf_country_curr', 'userf_brand_curr', 'userf']
if_names = ['None', 'itemf_group2', 'itemf_move', 'itemf_cost', 'itemf_group2_move', 'itemf_group2_cost', 'itemf_move_cost', 'itemf']

results = []
for u in range(len(uf_combi)):
    for i in range(len(if_combi)):
        res = cv.hyperp_tuning_simple(test=val_w, train=cal_w, seed=22, param_space=space_FM_warp, model_class='FM', user_features=uf_combi[u], item_features=if_combi[i])
        res['uf_name'] = uf_names[u]
        res['if_name'] = if_names[i]

        results.append(res)

In [105]:
results_df_tuned_add = unlist_df(results)

In [106]:
results_df_tuned = pd.concat([results_df_tuned, results_df_tuned_add], axis=0)

In [107]:
results_df_tuned.to_csv('fm_tuning_t.csv')

In [108]:
results_df_tuned.sort_values('precision', ascending=False).head(20)

,factors,regularization,learning_rate,iterations,loss,max_sampled,precision,map,ndcg,auc,mpr,uf_name,if_name
0,64,0.0003,0.010,15,warp,35,0.3350,0.186261,0.337689,0.510809,0.135406,userf,itemf
2,64,0.0010,0.010,15,warp,35,0.3300,0.204685,0.355762,0.510071,0.133574,userf_country_brand,itemf_group2_move
7,64,0.0003,0.010,15,warp,30,0.3300,0.181496,0.338202,0.509908,0.131905,userf_country,itemf_group2_move
25,64,0.0010,0.010,15,warp,30,0.3275,0.180028,0.332663,0.510032,0.133969,userf_country,itemf
25,64,0.0010,0.010,15,warp,30,0.3200,0.162761,0.320908,0.510276,0.138698,userf,itemf_move
17,64,0.0007,0.010,15,warp,40,0.3200,0.194071,0.345847,0.509402,0.133701,userf_country_curr,itemf
26,64,0.0010,0.010,15,warp,40,0.3175,0.179156,0.330351,0.509072,0.135442,userf_country_curr,itemf
26,64,0.0010,0.010,15,warp,40,0.3175,0.185123,0.342469,0.509668,0.134191,userf,itemf_group2_move
26,64,0.0010,0.010,15,warp,40,0.3150,0.173913,0.328406,0.509740,0.133007,userf_country,itemf_group2_move
0,64,0.0003,0.010,15,warp,35,0.3150,0.175798,0.336973,0.509672,0.133520,userf_country_brand,itemf


Then with BPR:

In [110]:
# step 2: take parameters and tune over all feature combinations lrtemp: 0.003, 0.007, 0.01, to add with regtemp: 0.0001
space_FM_bpr = {'factors' : [64], 'regularization' : [0.0001, 0.0003, 0.0007, 0.001], 'learning_rate' : [0.003, 0.007, 0.01, 0.03], 'iterations' : [15], 'loss' : ['bpr'], 'max_sampled':[1]}

uf_combi = [None, userf_country, userf_brand, userf_curr, userf_country_brand, userf_country_curr, userf_brand_curr, userf]
if_combi = [None, itemf_group2, itemf_move, itemf_cost, itemf_group2_move, itemf_group2_cost, itemf_move_cost, itemf]

uf_names = ['None', 'userf_country', 'userf_brand', 'userf_curr', 'userf_country_brand', 'userf_country_curr', 'userf_brand_curr', 'userf']
if_names = ['None', 'itemf_group2', 'itemf_move', 'itemf_cost', 'itemf_group2_move', 'itemf_group2_cost', 'itemf_move_cost', 'itemf']

results_bpr = []
for u in range(len(uf_combi)):
    for i in range(len(if_combi)):
        res = cv.hyperp_tuning_simple(test=val_w, train=cal_w, seed=22, param_space=space_FM_bpr, model_class='FM', user_features=uf_combi[u], item_features=if_combi[i])
        res['uf_name'] = uf_names[u]
        res['if_name'] = if_names[i]

        results_bpr.append(res)

In [111]:
results_df_bpr = unlist_df(results_bpr)

In [113]:
results_df_bpr

,factors,regularization,learning_rate,iterations,loss,max_sampled,precision,map,ndcg,auc,mpr,uf_name,if_name
0,64,0.0001,0.003,15,bpr,1,0.2650,0.128215,0.270529,0.508246,0.170894,None,None
1,64,0.0001,0.007,15,bpr,1,0.2525,0.127246,0.261420,0.506986,0.137238,None,None
2,64,0.0001,0.010,15,bpr,1,0.2525,0.130783,0.267986,0.507156,0.126810,None,None
3,64,0.0001,0.030,15,bpr,1,0.2050,0.116108,0.234234,0.505512,0.127833,None,None
4,64,0.0003,0.003,15,bpr,1,0.2525,0.125563,0.263107,0.507711,0.171937,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,64,0.0007,0.030,15,bpr,1,0.2075,0.094659,0.208582,0.505390,0.128111,userf,itemf
12,64,0.0010,0.003,15,bpr,1,0.1325,0.056502,0.143843,0.503215,0.200228,userf,itemf
13,64,0.0010,0.007,15,bpr,1,0.2275,0.115294,0.246866,0.506739,0.134617,userf,itemf
14,64,0.0010,0.010,15,bpr,1,0.2400,0.119616,0.246830,0.507362,0.128041,userf,itemf


In [114]:
results_df_bpr.to_csv('fm_tuning_t.csv')